Model is the lactose pathway. lactose is passively diffused through porin pores, active uptake by lac Permease, and then hydrolysis by beta-Galactosidase. 

In [31]:
import tellurium as te
print ("success")

Make sure you understand what you're plotting. The fundamental variable that gets changed is time because this is a kinetic modeling package.

In [38]:
model = """
model test
    compartment C1;
    C1 = 1.0;
    species S1, S2;
    S1 = 100; S2= 0;
    J1: S1 -> S2; (kcat*Etot*S1)/(km+S1);
    km = 5.5; #mM
    kcat = 10; #1/s
    Etot = 10;
    end
    """
r = te.loada(model)
r.simulate(0, 1, 1000, ["time","J1"]) #from, to, steps
r.plot()



Instead of doing the model within a python cell, it's better to use a model cell and get in the habit of using that. It's easier to not have to reload the entire model to plot or do any manipulations. It also allows you to order stuff in whatever order. It cross checks to make sure all the variables are there so you don't have to work completely linearly.

-> means irreversible

=> expect reversible rate law.

you can calculate the steady state values 
values = LacPath.getSteadyStateValues()
selections = LacPath.steadyStateSelections

print(values)

print(selections)

Because in a tiny cell, the concentrations are small, they'll perturb quickly. So steady state will occur very rapidly. So we'll mostly worry about what happens at steady state. 


In [ ]:
model 
    compartment C2, C3;
    
    species S3 in C3
    
    J34: S3 -> ; kcat_34*Etot_34*S3 / (Km_34 + S3)
    kcat_34 = 6.42e2; #S^-1; from Juers et al (2012)
    Etot_34 = 1.0e-3 ; #M, free parameter
    Km_34 = 1.01e1 ; # M; from BRENDA 5.5e-3
    
    S3 = 100;
    
    end


    
    
    
    
    
    
    
    
    
    
    
    